In [ ]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_images(data_dir, image_size=(224, 224)):
    images = []
    labels = []

    class_labels = os.listdir(data_dir)
    label_encoder = LabelEncoder()
    label_encoder.fit(class_labels)

    for label in class_labels:
          class_path = os.path.join(data_dir, label)

          for filename in os.listdir(class_path):
              image_path = os.path.join(class_path, filename)

              image = cv2.imread(image_path)
              image = cv2.resize(image, image_size)
              image = image / 255.0

              images.append(image)
              labels.append(label)

    images = np.array(images)
    labels = label_encoder.transform(labels)

    return images, labels


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/MyDrive/Hearbeat_Data_Test(100)"
images, labels = load_and_preprocess_images(data_dir)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(None, None, 3)))
# model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.2))

# Global Average Pooling
model.add(layers.GlobalMaxPooling2D())

# Dense layers
model.add(layers.Dense(32, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))


optimizer = optimizers.Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.3)
test_loss, test_accuracy = model.evaluate(X_test, y_test)


In [ ]:
model.save("Heartbeat-check-try1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_json = model.to_json()
with open("Heartbeat-check-try1", "w") as json_file:
    json_file.write(model_json)

# Optionally, save only the learned weights to a file
model.save_weights("Heartbeat-check-try1_weights.h5")

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model("Heartbeat-check-try1 (2).h5")
test_loss, test_accuracy = loaded_model.evaluate(images, labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

7/7 [==============================] - 4s 578ms/step - loss: 0.0426 - accuracy: 0.9800
Test Loss: 0.042610734701156616
Test Accuracy: 0.9800000190734863
